# Instantaneous firing rate

We calculate the instantaneous firing rate of all neurons and save the results for later.

For convenience, we should get the same time resolution as the AnimalPose.pose data (50Hz).


In [1]:
%load_ext autoreload
%autoreload 2

%run ~/repo/autopi_analysis_bk/Jazi_et.al_2023_noInt/setup_project.py
%run ~/repo/autopi_analysis_bk/Jazi_et.al_2023_noInt/neuronAutopi.py

prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList)

Project name: autopi_ca1
dataPath: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1
dlcModelPath: /adata/models
Reading /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:13<00:00,  2.99it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [2]:
ses = myProject.sessionList[-1]
sSes = sSesList[-1]

In [3]:
fn = ses.path+"/navPathInstan.csv"
print("loading",fn)
pd.read_csv(fn)

loading /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/mn9686/mn9686-01112021-0106/navPathInstan.csv


,name,trialNo,timeRos,timeRes,iTime,iTimeProp,distance,distanceProp,speed,x,y,targetDistance,targetToAnimalX,targetToAnimalY,targetToAnimalAngle,cumSumDiffAngleAroundTarget
0,mn9686-01112021-0106_1-0_all,1,1.635781e+09,3057.72,0.000000,0.000000,0.226477,0.000939,10.469781,-1.547535,-42.772820,43.748974,-0.886454,-49.407187,-1.588736,NaN
1,mn9686-01112021-0106_1-0_all,1,1.635781e+09,3057.74,0.021631,0.002355,0.521394,0.002163,11.984293,-1.505792,-42.550224,43.525129,-0.844711,-49.184590,-1.587969,0.000767
2,mn9686-01112021-0106_1-0_all,1,1.635781e+09,3057.76,0.046240,0.005035,0.817749,0.003392,13.761295,-1.477743,-42.256644,43.230798,-0.816662,-48.891010,-1.587499,0.001238
3,mn9686-01112021-0106_1-0_all,1,1.635781e+09,3057.78,0.067775,0.007380,1.209617,0.005018,20.032589,-1.470911,-41.960367,42.934455,-0.809830,-48.594734,-1.587460,0.001276
4,mn9686-01112021-0106_1-0_all,1,1.635781e+09,3057.80,0.087337,0.009510,1.694762,0.007030,30.082162,-1.502748,-41.569795,42.545125,-0.841667,-48.204161,-1.588255,0.000481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172877,mn9686-01112021-0106_160-1_atLever,160,1.635786e+09,8393.02,1.858261,0.950462,64.690139,0.965279,35.879708,-18.847819,-25.363831,8.941222,-15.222911,-1.085322,-3.070418,4.739598
172878,mn9686-01112021-0106_160-1_atLever,160,1.635786e+09,8393.04,1.879748,0.961452,65.473458,0.976967,36.726040,-18.781886,-26.131971,9.217626,-15.156978,-1.853462,-3.019912,4.790103
172879,mn9686-01112021-0106_160-1_atLever,160,1.635786e+09,8393.06,1.901077,0.972361,66.251160,0.988572,36.775143,-18.668076,-26.906977,9.511755,-15.043168,-2.628468,-2.968611,4.841405
172880,mn9686-01112021-0106_160-1_atLever,160,1.635786e+09,8393.08,1.922224,0.983178,67.017058,1.000000,23.287125,-18.505967,-27.667596,9.812199,-14.881058,-3.389087,-2.917667,4.892349


In [4]:
sSes.ap.intervals.inter
a = np.arange(0.01,8465.98,0.02)
print("a:", a.shape,a.min(),a.max())
print("pose:",sSes.ap.pose.shape,sSes.ap.pose[:,0].min(),sSes.ap.pose[:,0].max())
myDiff = np.diff(a)
print("diff:",myDiff.min(),myDiff.max())


a: (423299,) 0.01 8465.97
pose: (423249, 8) 0.0 8464.96
diff: 0.019999999998617568 0.020000000000436557


We need to set all spike_train objects to the same time interval that covers all values of the AnimalPose.pose. The idea is to get a firing rate estimate for each position data, which are at 50 Hz.

In [5]:
def createInstantaneousFiringRateFile(sSes):
    inter = sSes.ap.intervals.inter.copy()
    inter[0,0]=0.01 # so that the times of ifr windows matches the ap.pose times
    print(inter)
    for n in sSes.cg.neuron_list:
        n.spike_train.set_intervals(inter)
    sSes.cg.instantaneous_firing_rate(bin_size_sec = 0.02, sigma = 1,free_memory=True)
    ifr,mids = sSes.cg.ifr
        
    
    myDiff = np.diff(mids)
    print("time range:",mids.min(),mids.max())
    print("time interval min and max:",myDiff.min(),myDiff.max())
    
    fn = sSes.path+"/ifr_autopi.pkl"
    print("saving",fn)
    with open(fn, 'wb') as outp:
        pickle.dump((ifr,mids), outp, pickle.HIGHEST_PROTOCOL)
    # to allow garbage collection, and avoid crashing out of RAM
    sSes.cg.ifr = None

In [6]:
createInstantaneousFiringRateFile(sSes)

[[1.00000e-02 8.46596e+03]]


57it [00:00, 64.17it/s]


time range: 0.02 8465.96
time interval min and max: 0.019999999998617568 0.020000000000436557
saving /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/mn9686/mn9686-01112021-0106/ifr_autopi.pkl


## For all the sessions

In [7]:
for i,sSes in enumerate(sSesList):
    print(i,sSes.name)
    createInstantaneousFiringRateFile(sSes)

0 mn5824-20112020-0107
[[1.000000e-02 1.026168e+04]]


11it [00:00, 49.43it/s]


time range: 0.02 10261.68
time interval min and max: 0.019999999998617568 0.020000000000436557
saving /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/mn5824/mn5824-20112020-0107/ifr_autopi.pkl
1 mn5824-22112020-0107
[[1.0000e-02 1.0229e+04]]


22it [00:00, 53.86it/s]


time range: 0.02 10229.0
time interval min and max: 0.019999999998617568 0.020000000000436557
saving /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/mn5824/mn5824-22112020-0107/ifr_autopi.pkl
2 mn5824-24112020-0107
[[1.00000e-02 9.85056e+03]]


...

time range: 0.02 10555.98
time interval min and max: 0.019999999998617568 0.020000000000436557
saving /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/mn9686/mn9686-28102021-0107/ifr_autopi.pkl
37 mn9686-29102021-0106
[[1.00000e-02 9.67986e+03]]


65it [00:01, 54.29it/s]


time range: 0.02 9679.86
time interval min and max: 0.019999999998617568 0.020000000000436557
saving /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/mn9686/mn9686-29102021-0106/ifr_autopi.pkl
38 mn9686-01112021-0106
[[1.00000e-02 8.46596e+03]]


57it [00:00, 62.07it/s]


time range: 0.02 8465.96
time interval min and max: 0.019999999998617568 0.020000000000436557
saving /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/mn9686/mn9686-01112021-0106/ifr_autopi.pkl
